<a href="https://colab.research.google.com/github/adienlopez/EAN13-Barcode-Generator/blob/main/Webuy_Complete_DVD_Listing_Scraper_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Enhanced Webuy Complete DVD Listing Scraper with Resume and Checkpoint Functionality

Features:
- Checkpoint system with dual storage (Google Sheets + JSON)
- Resume from last position
- Append mode (never clears existing data)
- Robust error handling and deduplication
"""

!pip install selenium gspread google-auth oauth2client --quiet

import time
import re
import random
import json
import os
import gspread
from google.colab import auth
from google.auth import default
from urllib.parse import urlparse, parse_qs, quote
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime

# === CONFIGURATION ===
SPREADSHEET_ID = "1rRa8QKtkE0NmN3DVF2SeUJfMlvkW51YribIsTtscb68"
SHEET_NAME = "DVD"
CHECKPOINT_SHEET = "Checkpoints"
JSON_CHECKPOINT_FILE = "/content/dvd_checkpoint.json"
AGE_RATINGS = ["U", "PG", "12", "15", "18", "E", "tc"]
MAX_RETRIES = 2
CHECKPOINT_FREQUENCY = 10  # Save checkpoint every 10 pages

# === GOOGLE SHEETS SETUP ===
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
sheet = gc.open_by_key(SPREADSHEET_ID)

# === CHECKPOINT FUNCTIONS ===
def load_checkpoint_from_sheets():
    """Load checkpoint from Google Sheets"""
    try:
        checkpoint_ws = sheet.worksheet(CHECKPOINT_SHEET)
        records = checkpoint_ws.get_all_records()
        if records:
            latest = records[-1]  # Get most recent checkpoint
            return {
                'year': int(latest['year']),
                'rating': str(latest['rating']),  # Convert to string to match AGE_RATINGS
                'page': int(latest['page']),
                'scraped_count': int(latest['scraped_count']),
                'timestamp': latest['timestamp']
            }
    except Exception as e:
        print(f"⚠️ Error loading checkpoint from sheets: {e}")
    return None

def load_checkpoint_from_json():
    """Load checkpoint from JSON file"""
    try:
        if os.path.exists(JSON_CHECKPOINT_FILE):
            with open(JSON_CHECKPOINT_FILE, 'r') as f:
                data = json.load(f)
                # Ensure rating is a string
                data['rating'] = str(data['rating'])
                return data
    except Exception as e:
        print(f"⚠️ Error loading checkpoint from JSON: {e}")
    return None

def save_checkpoint(year, rating, page, scraped_count):
    """Save checkpoint to both Google Sheets and JSON"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    checkpoint_data = {
        'year': year,
        'rating': rating,
        'page': page,
        'scraped_count': scraped_count,
        'timestamp': timestamp
    }

    # Save to JSON (local backup)
    try:
        with open(JSON_CHECKPOINT_FILE, 'w') as f:
            json.dump(checkpoint_data, f)
    except Exception as e:
        print(f"⚠️ Failed to save JSON checkpoint: {e}")

    # Save to Google Sheets
    try:
        try:
            checkpoint_ws = sheet.worksheet(CHECKPOINT_SHEET)
        except gspread.WorksheetNotFound:
            # Create checkpoint sheet if it doesn't exist
            checkpoint_ws = sheet.add_worksheet(title=CHECKPOINT_SHEET, rows="100", cols="10")
            headers = ["year", "rating", "page", "scraped_count", "timestamp"]
            checkpoint_ws.append_row(headers)

        checkpoint_ws.append_row([year, rating, page, scraped_count, timestamp])
    except Exception as e:
        print(f"⚠️ Failed to save Google Sheets checkpoint: {e}")

def get_resume_info():
    """Try to load checkpoint and get user input for resume"""
    # Try Google Sheets first
    checkpoint = load_checkpoint_from_sheets()
    if not checkpoint:
        # Fallback to JSON
        checkpoint = load_checkpoint_from_json()

    if checkpoint:
        print(f"\n🔄 Found checkpoint:")
        print(f"   Year: {checkpoint['year']}")
        print(f"   Rating: {checkpoint['rating']}")
        print(f"   Last completed page: {checkpoint['page']}")
        print(f"   Items scraped: {checkpoint['scraped_count']}")
        print(f"   Timestamp: {checkpoint['timestamp']}")

        resume_choice = input(f"\nPress Enter to resume from here, or type 'new' for fresh start: ").strip()

        if resume_choice.lower() != 'new':
            # Resume from checkpoint
            return {
                'start_year': checkpoint['year'],
                'end_year': checkpoint['year'],  # Will be updated below
                'resume_rating': checkpoint['rating'],
                'resume_page': checkpoint['page'] + 1,
                'scraped_count': checkpoint['scraped_count']
            }

    # Fresh start or no checkpoint found
    print("\n🆕 Starting fresh scrape")
    start_year = int(input("🔢 Enter the START year (e.g. 2025): "))
    end_year = int(input("🔻 Enter the END year (e.g. 2010): "))

    return {
        'start_year': start_year,
        'end_year': end_year,
        'resume_rating': None,
        'resume_page': 1,
        'scraped_count': 0
    }

# === INITIALIZE RESUME INFO ===
resume_info = get_resume_info()
start_year = resume_info['start_year']
end_year = resume_info['end_year']

# If resuming, ask for end year
if resume_info['resume_rating']:
    end_year = int(input(f"🔻 Enter the END year (currently resuming {start_year}): "))

YEARS = list(range(start_year, end_year - 1, -1))
scraped_count = resume_info['scraped_count']

# === SETUP WORKSHEETS ===
# Main DVD worksheet - create if doesn't exist, but never clear
try:
    worksheet = sheet.worksheet(SHEET_NAME)
    print(f"📋 Using existing '{SHEET_NAME}' worksheet (appending data)")
except gspread.WorksheetNotFound:
    worksheet = sheet.add_worksheet(title=SHEET_NAME, rows="1000", cols="10")
    headers = ["Title", "Product ID", "Category", "Super Category", "Price", "Image URL"]
    worksheet.append_row(headers)
    print(f"📋 Created new '{SHEET_NAME}' worksheet")

# === SELENIUM SETUP ===
def create_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--user-agent=Mozilla/5.0")
    driver = webdriver.Chrome(options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver

# === FALLBACK IMAGE URL BUILDER ===
def construct_image_url(product_id, category_name):
    encoded_path = quote(category_name, safe='')
    return f"https://uk.static.webuy.com/product_images/DVD/{encoded_path}/{product_id}_l.jpg"

# === PRODUCT LINKS ===
def get_product_links(driver):
    try:
        # Wait for either products to appear OR for 'no results' to show
        WebDriverWait(driver, 30).until(
            EC.any_of(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='product-detail']")),
                EC.presence_of_element_located((By.CSS_SELECTOR, ".noResults, .notFound, .search-no-results"))
            )
        )
        time.sleep(2.5)

        try:
            elements = driver.find_elements(By.CSS_SELECTOR, "a[href*='product-detail']")
            links = [e.get_attribute("href") for e in elements if e.get_attribute("href")]
        except Exception:
            time.sleep(1)
            elements = driver.find_elements(By.CSS_SELECTOR, "a[href*='product-detail']")
            links = [e.get_attribute("href") for e in elements if e.get_attribute("href")]

        if not links:
            print("  ⚠️ No product links found. Dumping page source snippet for review:")
            print(driver.page_source[:1000])

        return list(set(links))
    except Exception as e:
        print(f"  ⚠️ Failed to load product links: {e}")
        return []

# === SCRAPER LOGIC ===
def extract_product_id(url):
    return parse_qs(urlparse(url).query).get("id", [None])[0]

def scrape_product_data(driver, url, retries=0):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "main")))
        time.sleep(random.uniform(1.5, 3.0))

        product_id = extract_product_id(url)
        title = "N/A"
        price = "N/A"

        # Extract title
        for sel in [".product-title", "h1", ".title"]:
            try:
                elem = driver.find_element(By.CSS_SELECTOR, sel)
                title = elem.text.strip()
                if title:
                    break
            except:
                continue

        # Extract price - FIXED REGEX PATTERN
        for elem in driver.find_elements(By.XPATH, "//*[contains(text(), '£')]"):
            text = elem.text.strip()
            match = re.search(r'£\d+(\.\d{2})?', text)
            if match:
                price = match.group()
                break

        # Extract categories from URL
        query = parse_qs(urlparse(url).query)
        cat = query.get("categoryName", ["Unknown"])[0]
        supercat = query.get("superCatName", ["Unknown"])[0]

        # Extract image URL directly from product page
        try:
            image_element = driver.find_element(By.CSS_SELECTOR, "img[src*='product_images']")
            image_url = image_element.get_attribute("src")
        except:
            image_url = construct_image_url(product_id, cat)

        return [title, product_id, cat, supercat, price, image_url]
    except Exception as e:
        if retries < MAX_RETRIES:
            time.sleep(10)
            return scrape_product_data(driver, url, retries + 1)
        return None

# === MAIN LOOP WITH RESUME LOGIC ===
driver = create_driver()
row_buffer = []
seen_ids = set()  # For current session deduplication
pages_processed = 0

# Determine where to start based on resume logic
start_rating_index = 0
start_page = 1
should_resume = False

if resume_info['resume_rating']:
    try:
        # Find the rating in our AGE_RATINGS list
        resume_rating = str(resume_info['resume_rating'])  # Ensure it's a string
        start_rating_index = AGE_RATINGS.index(resume_rating)
        start_page = resume_info['resume_page']
        should_resume = True
        print(f"🔄 Successfully set resume point: Rating '{resume_rating}' (index {start_rating_index}), Page: {start_page}")
    except ValueError:
        print(f"⚠️ Rating '{resume_info['resume_rating']}' not found in AGE_RATINGS {AGE_RATINGS}")
        print("⚠️ Available ratings are:", AGE_RATINGS)
        print("⚠️ Starting fresh from beginning")
        should_resume = False
    except Exception as e:
        print(f"⚠️ Error setting up resume: {e}")
        should_resume = False

for year in YEARS:
    # Skip years that are before our resume year
    if should_resume and year > resume_info['start_year']:
        continue

    # If this is the resume year, start from the correct rating
    rating_start_idx = start_rating_index if (should_resume and year == resume_info['start_year']) else 0

    for rating_idx in range(rating_start_idx, len(AGE_RATINGS)):
        rating = AGE_RATINGS[rating_idx]
        print(f"🟢 Scraping DVDs for Year: {year}, Rating: {rating}")
        base_url = f"https://uk.webuy.com/search?stext=dvd&Age+Rating+(BBFC)={rating}&Year={year}"

        stop_pagination = False

        # Determine starting page
        if should_resume and year == resume_info['start_year'] and rating_idx == start_rating_index:
            current_start_page = start_page
            print(f"   📍 Resuming from page {current_start_page}")
        else:
            current_start_page = 1

        for page in range(current_start_page, 61):
            if stop_pagination:
                break

            # Handle page 1 separately (no page parameter)
            if page == 1:
                page_url = base_url
            else:
                page_url = base_url + f"&page={page}"

            for attempt in range(2):
                try:
                    print(f"     🌐 Loading: {page_url}")
                    driver.get(page_url)
                    time.sleep(random.uniform(1.5, 3.5))

                    product_links = get_product_links(driver)
                    print(f"   🔍 Page {page} - Found {len(product_links)} product links")

                    if not product_links:
                        stop_pagination = True
                        break

                    # Track new products found on this page
                    new_products_found = 0

                    # Process each product link
                    for link in product_links:
                        product_id = extract_product_id(link)

                        # Skip if already scraped in this session
                        if product_id in seen_ids:
                            continue

                        # This is a new product
                        new_products_found += 1
                        data = scrape_product_data(driver, link)
                        if data:
                            row_buffer.append(data)
                            seen_ids.add(product_id)
                            scraped_count += 1

                    # If no new products found, stop pagination
                    if new_products_found == 0:
                        print(f"  ⏹️  No new products on page {page} - stopping pagination for this year/rating")
                        stop_pagination = True
                        break

                    # Write to Google Sheets in batches
                    if len(row_buffer) >= 50:
                        worksheet.append_rows(row_buffer, value_input_option="USER_ENTERED")
                        print(f"  ✅ Wrote {len(row_buffer)} rows | Total scraped: {scraped_count}")
                        row_buffer.clear()

                    pages_processed += 1

                    # Save checkpoint every CHECKPOINT_FREQUENCY pages
                    if pages_processed % CHECKPOINT_FREQUENCY == 0:
                        save_checkpoint(year, rating, page, scraped_count)
                        print(f"  💾 Checkpoint saved at Year {year}, Rating {rating}, Page {page}")

                    # Longer delay every 10 pages
                    if page % 10 == 0:
                        time.sleep(random.uniform(8, 15))

                    break  # Successful, exit retry loop

                except Exception as e:
                    print(f"  ⚠️ Attempt {attempt+1} failed on page {page}: {e}")
                    time.sleep(10)

        # Write remaining buffered data and save checkpoint for this year/rating
        if row_buffer:
            worksheet.append_rows(row_buffer, value_input_option="USER_ENTERED")
            print(f"  ✅ Wrote remaining {len(row_buffer)} rows for Year {year}, Rating {rating}")
            row_buffer.clear()

        # Save checkpoint after completing each year/rating combination
        last_page = page if 'page' in locals() else current_start_page
        save_checkpoint(year, rating, last_page, scraped_count)

        # Reset resume flag after first use
        if should_resume and year == resume_info['start_year'] and rating_idx == start_rating_index:
            should_resume = False  # Only apply resume logic once

    # Reset start_rating_index for subsequent years (always start from beginning)
    start_rating_index = 0

# Clean up
driver.quit()

# Final checkpoint and summary
if YEARS and AGE_RATINGS:
    save_checkpoint(YEARS[-1], AGE_RATINGS[-1], 60, scraped_count)

print(f"\n🎉 SCRAPING COMPLETED!")
print(f"📊 Total items scraped in this session: {scraped_count}")
print(f"📄 Data saved to Google Sheets tab '{SHEET_NAME}'")
print(f"💾 Final checkpoint saved to both Google Sheets and JSON")

# Clean up checkpoint file
try:
    if os.path.exists(JSON_CHECKPOINT_FILE):
        os.remove(JSON_CHECKPOINT_FILE)
        print("🧹 Cleaned up local checkpoint file")
except:
    pass

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 19.6 MB/s eta 0:00:00

🔄 Found checkpoint:
   Year: 2015
   Rating: 12
   Last completed page: 21
   Items scraped: 6292
   Timestamp: 2025-07-23 13:18:10

Press Enter to resume from here, or type 'new' for fresh start: 
🔻 Enter the END year (currently resuming 2015): 2010
📋 Using existing 'DVD' worksheet (appending data)
🔄 Successfully set resume point: Rating '12' (index 2), Page: 22
🟢 Scraping DVDs for Year: 2015, Rating: 12
   📍 Resuming from page 22
     🌐 Loading: https://uk.webuy.com/search?stext=dvd&Age+Rating+(BBFC)=12&Year=2015&page=22
  ⚠️ Failed to load product links: Message: 

   🔍 Page 22 - Found 0 product links
🟢 Scraping DVDs for Year: 2015, Rating: 15
     🌐 Loading: https://uk.webuy.com/search?stext=dvd&Age+Rating+(BBFC)=15&Year=2015
   🔍 Page 1 - Found 17 product links
     🌐 Loading: https://uk.webuy.com/search?stext=dvd&Age+Rating